In [3]:
%load_ext autoreload
%autoreload 2

# -*- coding: utf-8 -*-
# Developed: Aug 2023
# Last update: Dec 2023
'''
Procedure for K0 search.
'''

import os
import shutil
import datetime as dt
import time
import glob
import numpy as np
import pickle
from JobCreation import Load_HeliosphericParameters, griglia_valori_k0, SubmitSims
from EvaluateSimAndFlux import GetRawMatrix, EvaluateFlux

DEBUG = True
NK0 = 400  #Number of K0 to test
SLEEP_TIME = 60
APP = "_FD006"
OUTPUT_DICT = {}

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
def load_simulation_list(app_name, debug=False):
    """
        Load the list of simulations.

        Parameters:
        app_name (str): the application name.
        debug (bool): whether to print debug information.

        Returns:
        sim_list (list): the list of simulations.
    """

    # Counters for excluded simulations (Added by me)
    excluded_sims_filter = 0
    excluded_sims_type = 0

    sim_list = []
    if debug:
        print(f"Loading simulations for application: {app_name}")

    for line in open(f"Simulations{app_name}.list").readlines():
        if line.startswith("#"):
            continue

        single_sim = [x.strip() for x in line.replace("\t", "").split("|")[:8]]
        if debug:
            print(f"Parsed simulation: {single_sim}")

        # if debug is true, it excludes all files obtaining a final empty list, so I inibited it (MG)
        # if debug:
        if False:
            if (float(single_sim[3]) < 20180101 or float(single_sim[3]) > 20180102):
                excluded_sims_filter += 1
            continue

        if "Electron" in single_sim[1] or "Positron" in single_sim[1]:
            if debug:
                excluded_sims_type += 1
            continue

        print(f"Adding simulation: {single_sim}")
        sim_list.append(single_sim)

        ions = single_sim[1].strip()
        if ions not in OUTPUT_DICT:
            OUTPUT_DICT[ions] = {}
            if debug:
                print(f"Added new ion type to OUTPUT_DICT: {ions}")

    if debug:
        print(f"Loaded simulation list:")
        for sim in sim_list:
            print(sim)
        print(f"Ecluded simulations due to filter: {excluded_sims_filter}")
        print(f"Ecluded simulations due to particle type: {excluded_sims_type}")

    return sim_list

In [6]:
def initialize_output_dict(sim_list, x_exp, f_exp, er_exp_inf, er_exp_sup, k0_ref, k0_ref_err, debug=False):
    """
        Initialize the output dictionary with experimental data.

        Parameters:
        sim_list (list): the list of simulations.
        x_exp (np.array): the experimental rigidity values.
        f_exp (np.array): the experimental flux values.
        er_exp_inf (np.array): the experimental flux lower error values.
        er_exp_sup (np.array): the experimental flux upper error values.
        k0_ref (float): the reference K0 value.
        k0_ref_err (float): the reference K0 error value.
        debug (bool): whether to print debug information.

        Returns:
        None
    """

    # debug = True

    for el in sim_list:
        ions = el[1].strip()
        time = dt.datetime.strptime(el[3], '%Y%m%d').date()
        if debug:
            print(f"Processing simulation for ion: {ions}, date: {time}")

        # For each rigidity value in the experimental data (in the range [3, 11])
        for i, T in enumerate(x_exp):
            if not (3 <= T <= 11):
                continue

            if debug:
                print(f"Processing rigidity value: {T}")

            # Initialize the output dictionary
            if T not in OUTPUT_DICT[ions]:
                OUTPUT_DICT[ions][T] = {}
                if debug:
                    print(f"Initialized OUTPUT_DICT for ion: {ions}, rigidity: {T}")

            # Initialize the output dictionary for the current ion and rigidity value
            OUTPUT_DICT[ions][T][time] = {
                "diffBest": 9e99,
                "K0best": 0,
                "K0Min": 9e99,
                "K0Max": 0,
                "Fluxbest": 0,
                "FluxMin": 9e99,
                "FluxMax": 0,
                "fExp": f_exp[i],
                "ErExp_inf": er_exp_inf[i],
                "ErExp_sup": er_exp_sup[i],
                "K0ref": k0_ref,
                "K0Err_ref": k0_ref_err * k0_ref,
            }
            if debug:
                print(f"Initialized data for ion: {ions}, rigidity: {T}, date: {time}")

    if debug:
        print("Final OUTPUT_DICT:")
        for ion, data in OUTPUT_DICT.items():
            print(f"Ion: {ion}, Data: {data}")

In [7]:
def wait_for_files(output_dir_path_name, n_files_expected, sleep_time, debug=False):
    """
        Wait until the expected number of output files is present.

        Parameters:
        output_dir_path_name (str): the path to the output directory.
        n_files_expected (int): the number of files expected.
        sleep_time (int): the time to sleep between checks.
        debug (bool): whether to print debug information.

        Returns:
        None
    """
    print(f"Waiting for {n_files_expected} files in {output_dir_path_name}...")
    while len(glob.glob(output_dir_path_name)) < n_files_expected:
        time.sleep(sleep_time)
        if debug:
            print(f"Found {len(glob.glob(output_dir_path_name))} files so far.")

In [8]:
def process_simulations(el_sim_list, raw_matrix_sims, x_exp, debug=False):
    """
        Process simulation results to find the best K0 values.

        Parameters:
        el_sim_list (list): the list of simulations.
        raw_matrix_sims (dict): the raw matrix of simulation results.
        x_exp (np.array): the experimental rigidity values.
        debug (bool): whether to print debug information.

        Returns:
        None
    """
    ions = el_sim_list[1].strip()
    time = dt.datetime.strptime(el_sim_list[3], '%Y%m%d').date()
    if debug:
        print(f"Processing simulations for ion: {ions}, date: {time}")

    fluxes = EvaluateFlux(el_sim_list, raw_matrix_sims, x_exp, debug)

    # For each K0 value and simulated flux value
    for k0, (sim_en_rig, sim_flux) in fluxes.items():
        if debug:
            print(f"Evaluating K0: {k0}")

        # For each rigidity value in the experimental data (in the range [3, 11])
        for i, F in enumerate(sim_flux):
            T = x_exp[i]
            if not (3 <= T <= 11):
                continue

            ref_entry = OUTPUT_DICT[ions][T][time]
            diff_val = F - ref_entry['fExp']
            if debug:
                print(
                    f"Rigidity: {T}, Simulated Flux: {F}, Experimental Flux: {ref_entry['fExp']}, Difference: {diff_val}")

            if -ref_entry['ErExp_inf'] <= diff_val <= ref_entry['ErExp_sup']:
                if debug:
                    print(
                        f"Difference {diff_val} is within error bounds [{-ref_entry['ErExp_inf']}, {ref_entry['ErExp_sup']}]")

                # If the difference is better than the current best difference, update the reference entry
                if abs(diff_val) < abs(ref_entry['diffBest']):
                    ref_entry['diffBest'] = diff_val
                    ref_entry['K0best'] = k0
                    ref_entry['Fluxbest'] = F
                    if debug:
                        print(f"Updated best values for ion: {ions}, rigidity: {T}, date: {time}")

                # If the K0 value is better than the current best K0 value, update the reference entry
                if ref_entry['K0Min'] > k0:
                    ref_entry['FluxMin'] = F
                    ref_entry['K0Min'] = k0
                    if debug:
                        print(f"Updated minimum K0 for ion: {ions}, rigidity: {T}, date: {time}")

                # If the K0 value is better than the current worst K0 value, update the reference entry
                if ref_entry['K0Max'] < k0:
                    ref_entry['FluxMax'] = F
                    ref_entry['K0Max'] = k0
                    if debug:
                        print(f"Updated maximum K0 for ion: {ions}, rigidity: {T}, date: {time}")

In [9]:
## Magic physics functions!


def smooth_transition(initial_val, final_val, center_of_transition, smoothness, x):
    # smooth transition between  InitialVal to FinalVal centered at CenterOfTransition as function of x
    # if smoothness== 0 use a sharp transition
    if smoothness == 0:
        return final_val if x >= center_of_transition else initial_val
    else:
        return (initial_val + final_val) / 2. - (initial_val - final_val) / 2. * np.tanh(
            (x - center_of_transition) / smoothness)


def k0_fit_ssn(p, solar_phase, ssn):
    if p > 0.:
        if solar_phase == 0:  # Rising
            k0 = 0.0002743 - 2.11e-6 * ssn + 1.486e-8 * ssn * ssn - 3.863e-11 * ssn * ssn * ssn;
            gauss_var = 0.1122
        else:  # Declining
            k0 = 0.0002787 - 1.66e-6 * ssn + 4.658e-9 * ssn * ssn - 6.673e-12 * ssn * ssn * ssn;
            gauss_var = 0.1324
    else:
        if solar_phase == 0:  # Rising
            k0 = 0.0003059 - 2.51e-6 * ssn + 1.284e-8 * ssn * ssn - 2.838e-11 * ssn * ssn * ssn;
            gauss_var = 0.1097
        else:  # Declining
            k0 = 0.0002876 - 3.715e-6 * ssn + 2.534e-8 * ssn * ssn - 5.689e-11 * ssn * ssn * ssn;
            gauss_var = 0.14
    return k0, gauss_var


def k0_fit_nmc(nmc):
    return np.exp(-10.83 - 0.0041 * nmc + 4.52e-5 * nmc * nmc), 0.1045


def k0_corr_factor(p, q, solar_phase, tilt):
    #   /*Authors: 2017 Stefano */
    #   /* * description: Correction factor to K0 for the Kparallel. This correction is introduced
    #                     to account for the fact that K0 is evaluated with a model not including particle drift.
    #                     Thus, the value need a correction once to be used in present model
    #       \param p            solar polarity of HMF
    #       \param q            signum of particle charge
    #       \param SolarPhase   0=rising / 1=Declining phase of solar activity cycle
    #       \param tilt         Tilt angle of neutral sheet (in degree)
    #   */
    k0_corr_maxv = 1.5
    k0_corr_minv = 1.
    k0_corr_p0_asc = 18.
    k0_corr_p1_asc = 40.
    k0_corr_p0_des = 5.
    k0_corr_p1_des = 53.
    k0_corr_maxv_neg = 0.7
    k0_corr_p0_asc_neg = 5.8
    k0_corr_p1_asc_neg = 47.
    k0_corr_p0_des_neg = 5.8
    k0_corr_p1_des_neg = 58.

    if q > 0:
        if q * p > 0:
            if solar_phase == 0:  # ascending
                return smooth_transition(k0_corr_maxv, k0_corr_minv, k0_corr_p1_asc, k0_corr_p0_asc, tilt)
            else:  # descending
                return smooth_transition(k0_corr_maxv, k0_corr_minv, k0_corr_p1_des, k0_corr_p0_des, tilt)
        else:
            return 1
    if q < 0:
        if q * p > 0:
            if solar_phase == 0:  # ascending
                return smooth_transition(k0_corr_maxv, k0_corr_minv, k0_corr_p1_asc, k0_corr_p0_asc, tilt)
            else:  # descending
                return smooth_transition(k0_corr_maxv, k0_corr_minv, k0_corr_p1_des, k0_corr_p0_des, tilt)
        else:
            if solar_phase == 0:  # ascending
                return smooth_transition(k0_corr_maxv_neg, k0_corr_minv, k0_corr_p1_asc_neg, k0_corr_p0_asc_neg, tilt)
            else:  # descending
                return smooth_transition(k0_corr_maxv_neg, k0_corr_minv, k0_corr_p1_des_neg, k0_corr_p0_des_neg, tilt)
    return 1


def eval_k0(is_high_activity_period, p, q, solar_phase, tilt, nmc, ssn):
    #   /*Authors: 2022 Stefano */
    #   /* * description: Evaluate diffusion parameter from fitting procedures.
    #       \param p            solar polarity of HMF
    #       \param q            signum of particle charge
    #       \param SolarPhase   0=rising / 1=Declining phase of solar activity cycle
    #       \param tilt         Tilt angle of neutral sheet (in degree)
    #       \return x = k0_paral
    #               y = k0_perp
    #               z = GaussVar
    #   */
    #   float3 output;
    k0cor = k0_corr_factor(p, q, solar_phase, tilt)  #; // k0_paral is corrected by a correction factor
    if is_high_activity_period and nmc > 0:
        k0, kerr = k0_fit_nmc(nmc)
    else:
        k0, kerr = k0_fit_ssn(p, solar_phase, ssn)
    return k0 * k0cor, kerr

In [10]:
def k0_grid(t: int, nk0: int, hpar: np.ndarray, q=+1, debug=False):
    """
        Generate a grid of K0 values.

        Parameters:
        t (int): the time step for each simulation.
        nk0 (int): the number of K0 values.
        heliospheric_parameters (list): the list of heliospheric parameters for each simulation.
        q (int): the sign of particle charge.
        debug (bool): whether to print debug information.

        Returns:
        k0grid (dict): the grid of K0 values.
        k0 (int): the reference k0 value.
        k0err (int): the error of the reference k0 value.
    """
    k0grid = np.ones(nk0)
    k0, k0err = -1, 1
    # Load the parameters and iterate each row
    for i in range(hpar.shape[1]):
        beg_cr = int(hpar[0][i])
        end_cr = int(hpar[1][i])
        if beg_cr <= t < end_cr:
            ssn = hpar[2][i]
            #V0          = Hpar[3][i]
            tilt_l = hpar[4][i]
            SmoothTiltL = hpar[12][i]
            #Bfield      = Hpar[6][i]
            polarity = hpar[7][i]
            solar_phase = hpar[8][i]
            nmcr = hpar[11][i]
            is_high_activity_period = (np.average([float(tilt) for tilt in hpar[4][i:i + 15]])) > 50
            k0, k0err = eval_k0(is_high_activity_period, polarity, q, solar_phase, tilt_l, nmcr, ssn)
            if debug:
                print(f"IsHighActivityPeriod {is_high_activity_period}")
                print(f"K0 {k0} +- {k0err * k0}")
            k0grid = np.linspace(k0 - 4 * k0err * k0, k0 + 4 * k0err * k0, num=nk0, endpoint=True)
            break
    return k0grid, k0, k0err

In [11]:
# EXECUTION FLOW
# Load simulation list
sim_list = load_simulation_list(APP, DEBUG)
# Load heliospheric parameters
hpar = Load_HeliosphericParameters(DEBUG)
len(sim_list), hpar.shape

Loading simulations for application: _FD006
Parsed simulation: ['AMS-02Daily_20110801', 'Helium', 'Rigidity_AMS-02Daily_20110801_Helium.dat', '20110801', '20110801', '1', '0', '0']
Adding simulation: ['AMS-02Daily_20110801', 'Helium', 'Rigidity_AMS-02Daily_20110801_Helium.dat', '20110801', '20110801', '1', '0', '0']
Added new ion type to OUTPUT_DICT: Helium
Parsed simulation: ['AMS-02Daily_20110802', 'Helium', 'Rigidity_AMS-02Daily_20110802_Helium.dat', '20110802', '20110802', '1', '0', '0']
Adding simulation: ['AMS-02Daily_20110802', 'Helium', 'Rigidity_AMS-02Daily_20110802_Helium.dat', '20110802', '20110802', '1', '0', '0']
Parsed simulation: ['AMS-02Daily_20110803', 'Helium', 'Rigidity_AMS-02Daily_20110803_Helium.dat', '20110803', '20110803', '1', '0', '0']
Adding simulation: ['AMS-02Daily_20110803', 'Helium', 'Rigidity_AMS-02Daily_20110803_Helium.dat', '20110803', '20110803', '1', '0', '0']
Parsed simulation: ['AMS-02Daily_20110804', 'Helium', 'Rigidity_AMS-02Daily_20110804_Helium.

(26, (17, 966))

In [12]:
el_sim_list = sim_list[0]
k0_arr, k0_ref, k0_ref_err = k0_grid(int(el_sim_list[3]), NK0, hpar, DEBUG)
k0_arr

array([0.00011179, 0.00011219, 0.00011259, 0.00011299, 0.0001134 ,
       0.0001138 , 0.0001142 , 0.0001146 , 0.00011501, 0.00011541,
       0.00011581, 0.00011621, 0.00011662, 0.00011702, 0.00011742,
       0.00011782, 0.00011823, 0.00011863, 0.00011903, 0.00011943,
       0.00011984, 0.00012024, 0.00012064, 0.00012104, 0.00012145,
       0.00012185, 0.00012225, 0.00012265, 0.00012306, 0.00012346,
       0.00012386, 0.00012426, 0.00012467, 0.00012507, 0.00012547,
       0.00012587, 0.00012628, 0.00012668, 0.00012708, 0.00012748,
       0.00012788, 0.00012829, 0.00012869, 0.00012909, 0.00012949,
       0.0001299 , 0.0001303 , 0.0001307 , 0.0001311 , 0.00013151,
       0.00013191, 0.00013231, 0.00013271, 0.00013312, 0.00013352,
       0.00013392, 0.00013432, 0.00013473, 0.00013513, 0.00013553,
       0.00013593, 0.00013634, 0.00013674, 0.00013714, 0.00013754,
       0.00013795, 0.00013835, 0.00013875, 0.00013915, 0.00013956,
       0.00013996, 0.00014036, 0.00014076, 0.00014117, 0.00014

In [13]:
file_name = f"DataTXT/{el_sim_list[2]}"
x_exp, f_exp, er_exp_inf, er_exp_sup = np.loadtxt(file_name, unpack=True, usecols=(0, 1, 2, 3))
valid_indices = (x_exp >= 3) & (x_exp <= 11)
x_exp, f_exp = x_exp[valid_indices], f_exp[valid_indices]
er_exp_inf, er_exp_sup = er_exp_inf[valid_indices], er_exp_sup[valid_indices]
initialize_output_dict([el_sim_list], x_exp, f_exp, er_exp_inf, er_exp_sup, k0_ref, k0_ref_err, debug=DEBUG)

Processing simulation for ion: Helium, date: 2011-08-01
Processing rigidity value: 3.13
Initialized OUTPUT_DICT for ion: Helium, rigidity: 3.13
Initialized data for ion: Helium, rigidity: 3.13, date: 2011-08-01
Processing rigidity value: 3.46
Initialized OUTPUT_DICT for ion: Helium, rigidity: 3.46
Initialized data for ion: Helium, rigidity: 3.46, date: 2011-08-01
Processing rigidity value: 3.83
Initialized OUTPUT_DICT for ion: Helium, rigidity: 3.83
Initialized data for ion: Helium, rigidity: 3.83, date: 2011-08-01
Processing rigidity value: 4.22
Initialized OUTPUT_DICT for ion: Helium, rigidity: 4.22
Initialized data for ion: Helium, rigidity: 4.22, date: 2011-08-01
Processing rigidity value: 4.65
Initialized OUTPUT_DICT for ion: Helium, rigidity: 4.65
Initialized data for ion: Helium, rigidity: 4.65, date: 2011-08-01
Processing rigidity value: 5.12
Initialized OUTPUT_DICT for ion: Helium, rigidity: 5.12
Initialized data for ion: Helium, rigidity: 5.12, date: 2011-08-01
Processing rig

In [14]:
sim_name, ions, file_name, init_date, final_date, radius, latitude, longitude = el_sim_list
ions = [i.strip() for i in ions.split(',')]
ions_str = '_'.join(ions)
is_tko = "Rigidity" not in file_name
start_dir = os.getcwd()

start_dir
os.path.join(start_dir, f"{sim_name}_{ions_str}")


'/Users/stark/Development/Research/Cosmica-dev/extra/Simulations_K0Search/AMS-02Daily_20110801_Helium'

In [15]:
for el_sim_list in sim_list[:1]:
    print(f"Processing {el_sim_list[0]} {el_sim_list[1]}...")

    # Create grid of K0 values to simulate
    k0_arr, k0_ref, k0_ref_err = griglia_valori_k0(el_sim_list[3], NK0, hpar, DEBUG)
    if DEBUG:
        print(f"Generated K0 grid: {k0_arr}, reference K0: {k0_ref}, reference K0 error: {k0_ref_err}")

    # Load experimental data and initialize output dictionary
    file_name = f"DataTXT/{el_sim_list[2]}"
    x_exp, f_exp, er_exp_inf, er_exp_sup = np.loadtxt(file_name, unpack=True, usecols=(0, 1, 2, 3))
    valid_indices = (x_exp >= 3) & (x_exp <= 11)
    x_exp, f_exp = x_exp[valid_indices], f_exp[valid_indices]
    er_exp_inf, er_exp_sup = er_exp_inf[valid_indices], er_exp_sup[valid_indices]
    if DEBUG:
        print(f"Loaded experimental data from {file_name}")

    initialize_output_dict([el_sim_list], x_exp, f_exp, er_exp_inf, er_exp_sup, k0_ref, k0_ref_err, debug=DEBUG)

    # Submit simulations and wait for results
    output_dir_path, output_file_list = SubmitSims(el_sim_list, k0_arr, R_range=[3, 11])

    if DEBUG:
        print(f"Submitted simulations, output directory: {output_dir_path}, output files: {output_file_list}")

    wait_for_files(f"{output_dir_path}/outfile/*.dat", len(output_file_list), SLEEP_TIME, DEBUG)
    if DEBUG:
        print(f"All simulation files are ready in {output_dir_path}/outfile/")

    # Process results of simulations
    raw_matrix_sims = GetRawMatrix(output_dir_path, output_file_list, DEBUG)
    process_simulations(el_sim_list, raw_matrix_sims, x_exp, debug=DEBUG)

    # Cleanup output directory
    if output_dir_path and 'Simulations_K0Search' in output_dir_path:
        print(f"Removing directory {output_dir_path}.")
        shutil.rmtree(output_dir_path)

# Save results to file Result{APP}.pkl using pickle
with open(f"Result{APP}.pkl", "wb") as f:
    pickle.dump(OUTPUT_DICT, f, protocol=pickle.HIGHEST_PROTOCOL)
print(f"Results saved to Result{APP}.pkl")


Processing AMS-02Daily_20110801 Helium...
Generated K0 grid: [0.00011179 0.00011219 0.00011259 0.00011299 0.0001134  0.0001138
 0.0001142  0.0001146  0.00011501 0.00011541 0.00011581 0.00011621
 0.00011662 0.00011702 0.00011742 0.00011782 0.00011823 0.00011863
 0.00011903 0.00011943 0.00011984 0.00012024 0.00012064 0.00012104
 0.00012145 0.00012185 0.00012225 0.00012265 0.00012306 0.00012346
 0.00012386 0.00012426 0.00012467 0.00012507 0.00012547 0.00012587
 0.00012628 0.00012668 0.00012708 0.00012748 0.00012788 0.00012829
 0.00012869 0.00012909 0.00012949 0.0001299  0.0001303  0.0001307
 0.0001311  0.00013151 0.00013191 0.00013231 0.00013271 0.00013312
 0.00013352 0.00013392 0.00013432 0.00013473 0.00013513 0.00013553
 0.00013593 0.00013634 0.00013674 0.00013714 0.00013754 0.00013795
 0.00013835 0.00013875 0.00013915 0.00013956 0.00013996 0.00014036
 0.00014076 0.00014117 0.00014157 0.00014197 0.00014237 0.00014278
 0.00014318 0.00014358 0.00014398 0.00014438 0.00014479 0.00014519
 0.

KeyboardInterrupt: 